# This is the source code of the Iteration 4 for Info722 Assessment

All the code is coding on the jupyter notebook on AWS EC2 Instance services

Instance ID i-09e9cee8d00d748c2
Public IPv4 address 34.202.158.7 | open address 
Instance state Running
Public IPv4 DNS ec2-34-202-158-7.compute-1.amazonaws.com | open address 

## Step1 Data Preparing

This part is coding and testing twice, with the data file in the data folder located on AWS EC2 Instance, and the other is directly read the csv files via URL from the NZTA website

In [2]:
# This part is initial the pyspark envrionment

import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

# Import pyspark modules
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
spark = SparkSession.builder.appName('basics').getOrCreate()

# import the matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# import the pyspark function
import pyspark.sql.functions as fn

# import the date time functions 
from pyspark.sql.functions import dayofmonth,month,hour,year,format_number

import numpy


### Get the data from the csv files 
All the csv files are unloaded into ../data folder

#### Get the csv files directly from the NZTA website
This part will not include in the final run, because the performance issue, need wait long time for the data transfering between AWS data centre and NZTA webside

In [ ]:
# Initial the csv files location and file names
# This is just teting run to verify performance and stability 

# FileVehicleYear2015 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2015.csv"
# FileVehicleYear2016 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2016.csv"
# FileVehicleYear2017 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2017.csv"
# FileVehicleYear2018 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2018.csv"
# FileVehicleYear2019 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2019.csv"
# FileVehicleYear2020 = "https://nztaopendata.blob.core.windows.net/motorvehicleregister/VehicleYear-2020.csv"
# 
# # Import the public libs for the data frame and read data from csv files
# 
# from pyspark import SparkFiles
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# 
# #Get the file content via the URLs
# spark.sparkContext.addFile(FileVehicleYear2015)
# VehicleYear2015 = spark.read.option("header",True).csv(SparkFiles.get("VehicleYear-2015.csv"))
# 
# spark.sparkContext.addFile(FileVehicleYear2016)
# VehicleYear2016 = spark.read.option("header",True).csv(SparkFiles.get("VehicleYear-2016.csv"))
# 
# spark.sparkContext.addFile(FileVehicleYear2017)
# VehicleYear2017 = spark.read.option("header",True).csv(SparkFiles.get("VehicleYear-2017.csv"))
# 
# spark.sparkContext.addFile(FileVehicleYear2018)
# VehicleYear2018 = spark.read.option("header",True).csv(SparkFiles.get("VehicleYear-2018.csv"))
# 
# spark.sparkContext.addFile(FileVehicleYear2019)
# VehicleYear2019 = spark.read.option("header",True).csv(SparkFiles.get("VehicleYear-2019.csv"))

# Verify the performance 
# print(VehicleYear2015.columns)


#### Get the csv files directly from the local drivers
Upload the data files into the ./data folder, and pyspark will import the data from local drivers, and this will much faster than get the data via internet.

In [3]:
# Read the csv files from the local drivers 
# Initial the csv files location 

# Get the file path of the data files, this is the relavent path 
FileVehiclePath = "./data/"

# 2.1 Collect initial data
# Get the five vehciles data files which download from NZTA website
FileVehicleYear2015 = FileVehiclePath + "VehicleYear-2015.csv"
FileVehicleYear2016 = FileVehiclePath + "VehicleYear-2016.csv"
FileVehicleYear2017 = FileVehiclePath + "VehicleYear-2017.csv"
FileVehicleYear2018 = FileVehiclePath + "VehicleYear-2018.csv"
FileVehicleYear2019 = FileVehiclePath + "VehicleYear-2019.csv"

# Read in the CSV data.
VehicleYear2015 = spark.read.csv(FileVehicleYear2015,inferSchema=True,header=True)
VehicleYear2016 = spark.read.csv(FileVehicleYear2016,inferSchema=True,header=True)
VehicleYear2017 = spark.read.csv(FileVehicleYear2017,inferSchema=True,header=True)
VehicleYear2018 = spark.read.csv(FileVehicleYear2018,inferSchema=True,header=True)
VehicleYear2019 = spark.read.csv(FileVehicleYear2019,inferSchema=True,header=True)

# Check if the data is loaded, and file schemas are recognized
VehicleYear2015.printSchema()
VehicleYear2016.printSchema()
VehicleYear2017.printSchema()
VehicleYear2018.printSchema()
VehicleYear2019.printSchema()

root
 |-- ALTERNATIVE_MOTIVE_POWER: string (nullable = true)
 |-- BASIC_COLOUR: string (nullable = true)
 |-- BODY_TYPE: string (nullable = true)
 |-- CC_RATING: integer (nullable = true)
 |-- CHASSIS7: string (nullable = true)
 |-- CLASS: string (nullable = true)
 |-- ENGINE_NUMBER: string (nullable = true)
 |-- FIRST_NZ_REGISTRATION_YEAR: integer (nullable = true)
 |-- FIRST_NZ_REGISTRATION_MONTH: integer (nullable = true)
 |-- GROSS_VEHICLE_MASS: decimal(6,0) (nullable = true)
 |-- HEIGHT: integer (nullable = true)
 |-- IMPORT_STATUS: string (nullable = true)
 |-- INDUSTRY_CLASS: string (nullable = true)
 |-- INDUSTRY_MODEL_CODE: string (nullable = true)
 |-- MAKE: string (nullable = true)
 |-- MODEL: string (nullable = true)
 |-- MOTIVE_POWER: string (nullable = true)
 |-- MVMA_MODEL_CODE: string (nullable = true)
 |-- NUMBER_OF_AXLES: integer (nullable = true)
 |-- NUMBER_OF_SEATS: integer (nullable = true)
 |-- NZ_ASSEMBLED: string (nullable = true)
 |-- ORIGINAL_COUNTRY: string 

### Merge the 5 years datasets into one, all the data analysis will based on the merge dataset
To imporve the performance issue, and just in case if the data files provide by NZTA contains the duplication, will use the reduce function to remove the duplication via the VIN11 

In [4]:
# Import the data frame moduels and will use reduce to union/merger all dataset together
from functools import reduce 
from pyspark.sql import DataFrame

df_VehicleYear2015 = VehicleYear2015
df_VehicleYear2016 = VehicleYear2016
df_VehicleYear2017 = VehicleYear2017
df_VehicleYear2018 = VehicleYear2018
df_VehicleYear2019 = VehicleYear2019

df_VehicleMerge = [df_VehicleYear2015,df_VehicleYear2016,df_VehicleYear2017,df_VehicleYear2018,df_VehicleYear2019]
df_VehicleAll = reduce(DataFrame.unionAll,df_VehicleMerge)

#df_VehicleAll.show()
#Get the statistics of the final merge dataset size 
df_VehicleAll_rowcount = df_VehicleAll.count()
df_VehicleAll_colnumber = len(df_VehicleAll.dtypes)

#Get the statistics of the 2015 dataset size
df_VehicleYear2015_rowcount = df_VehicleYear2015.count()
df_VehicleYear2015_colnumber = len(df_VehicleYear2015.dtypes)

#Get the statistics of the 2016 dataset size
df_VehicleYear2016_rowcount = df_VehicleYear2016.count()
df_VehicleYear2016_colnumber = len(df_VehicleYear2016.dtypes)

#Get the statistics of the 2017 dataset size
df_VehicleYear2017_rowcount = df_VehicleYear2017.count()
df_VehicleYear2017_colnumber = len(df_VehicleYear2017.dtypes)

#Get the statistics of the 2018 dataset size
df_VehicleYear2018_rowcount = df_VehicleYear2018.count()
df_VehicleYear2018_colnumber = len(df_VehicleYear2018.dtypes)

#Get the statistics of the 2019 dataset size
df_VehicleYear2019_rowcount = df_VehicleYear2019.count()
df_VehicleYear2019_colnumber = len(df_VehicleYear2019.dtypes)

In [4]:
# Verify the sum of data row count for each year is the same as total 

print("Total count from 2015 - 2019: ",df_VehicleAll_rowcount)
print("Total count for 2015: ",df_VehicleYear2015_rowcount)
print("Total count for 2016: ",df_VehicleYear2016_rowcount)
print("Total count for 2017: ",df_VehicleYear2017_rowcount)
print("Total count for 2018: ",df_VehicleYear2018_rowcount)
print("Total count for 2019: ",df_VehicleYear2019_rowcount)
#

# df_VehicleYear2018.head(5)

# Check the data missing, and for the MOTIVE_POWER need to be fixed the missing records before process
df_VehicleAll.agg(*[(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing') for c in df_VehicleAll.columns]).show()

df_VehicleYear2018.where("MOTIVE_POWER is null").head()

Total count from 2015 - 2019:  1017971
Total count for 2015:  186749
Total count for 2016:  198389
Total count for 2017:  212335
Total count for 2018:  214225
Total count for 2019:  206273
+--------------------------------+--------------------+-----------------+-----------------+------------------+--------------------+---------------------+----------------------------------+-----------------------------------+--------------------------+--------------+---------------------+----------------------+---------------------------+------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+------------------------+--------------------+------------------------+---------------------------+------------------+-----------+-------------------------+-------------------+--------------------+---------------------+--------------------+-------------------+-------------+
|ALTERNATIVE_MOTIVE_POWER_missing|BASIC_COLOUR_mi

Row(ALTERNATIVE_MOTIVE_POWER=None, BASIC_COLOUR='WHITE', BODY_TYPE='DOMESTIC TRAILER', CC_RATING=0, CHASSIS7=None, CLASS='TB', ENGINE_NUMBER=None, FIRST_NZ_REGISTRATION_YEAR=2018, FIRST_NZ_REGISTRATION_MONTH=8, GROSS_VEHICLE_MASS=Decimal('2500'), HEIGHT=0, IMPORT_STATUS='SCRATCH', INDUSTRY_CLASS='UNKNOWN', INDUSTRY_MODEL_CODE=None, MAKE='HOMEBUILT', MODEL='TRAILER', MOTIVE_POWER=None, MVMA_MODEL_CODE=None, NUMBER_OF_AXLES=1, NUMBER_OF_SEATS=0, NZ_ASSEMBLED='NZ ASSEMBLED/BUILT', ORIGINAL_COUNTRY='NEW ZEALAND', POWER_RATING=0, PREVIOUS_COUNTRY='NONE', ROAD_TRANSPORT_CODE=None, SUBMODEL=None, TLA=None, TRANSMISSION_TYPE=None, VDAM_WEIGHT=0, VEHICLE_TYPE='TRAILER/CARAVAN', VEHICLE_USAGE='OTHER GOODS', VEHICLE_YEAR=2018, VIN11=None, WIDTH=0)

In [6]:
# Filter the non power vehicles
# Processing the data Quality
# Filter outer the missing value records on MOTIVE_POWER, all the "TRAILER" type vehciles, which 
# do not contain the engins 
# Row count before clean up

df_VehicleAll
#
print("Row count before clean up: " ,df_VehicleAll.count())
df_VehicleAll = df_VehicleAll.filter("MAKE <> \"TRAILER\"" and "MOTIVE_POWER <> \"\"" 
                            and "IMPORT_STATUS <> \"SCRATCH\"" 
                            and "FIRST_NZ_REGISTRATION_YEAR < \"2020\""
                            and "VEHICLE_TYPE not LIKE \"%TRAILER%\" ")

# Row count after clean up
print("Row count after clean up: " ,df_VehicleAll.count())



Row count before clean up:  841295
Row count after clean up:  841295


In [7]:
# 2.2 Describe the data
print (df_VehicleAll.head())
print (df_VehicleAll.describe())

Row(ALTERNATIVE_MOTIVE_POWER=None, BASIC_COLOUR='WHITE', BODY_TYPE='OTHER TRUCK', CC_RATING=4009, CHASSIS7='JHHNCT2', CLASS='NB', ENGINE_NUMBER='NO4C-UT15093', FIRST_NZ_REGISTRATION_YEAR=2015, FIRST_NZ_REGISTRATION_MONTH=3, GROSS_VEHICLE_MASS=Decimal('8500'), HEIGHT=2560, IMPORT_STATUS='NEW', INDUSTRY_CLASS='CONSTRUCTING', INDUSTRY_MODEL_CODE=None, MAKE='HINO', MODEL='300', MOTIVE_POWER='DIESEL', MVMA_MODEL_CODE=None, NUMBER_OF_AXLES=2, NUMBER_OF_SEATS=3, NZ_ASSEMBLED='IMPORTED BUILT-UP', ORIGINAL_COUNTRY='JAPAN', POWER_RATING=0, PREVIOUS_COUNTRY='NONE', ROAD_TRANSPORT_CODE=None, SUBMODEL='XZU730R-QKFTTQ3', TLA=None, TRANSMISSION_TYPE=None, VDAM_WEIGHT=14200, VEHICLE_TYPE='GOODS VAN/TRUCK/UTILITY', VEHICLE_USAGE='TRANSPORT LICENSED GOODS', VEHICLE_YEAR=2015, VIN11='JHHNCT2H70K', WIDTH=2220)
DataFrame[summary: string, ALTERNATIVE_MOTIVE_POWER: string, BASIC_COLOUR: string, BODY_TYPE: string, CC_RATING: string, CHASSIS7: string, CLASS: string, ENGINE_NUMBER: string, FIRST_NZ_REGISTRATION

### Infosys 722 Python Assessment Iteration 4  Data Preparing
Adding the Promotion column

In [8]:
#Infosys 722 Python Assessment Iteration 4  Data Preparing
#
#4. Data Transformation
#4.1    Data Reduce and resharp
#This is reference in the reference section 8
#Based on the Ministry of Trannsport of New Zealand website, there is govement promotion programme 
#which encourage consumers to choose electric vehicles. The key importance of the promotion is the 
#exemption of the Road user cost, from that website, officially announced that buying a electric 
#vehicles will save appromaxmately $600 per year per vehicles, in Using the derived stage to adding 
#the EV and from 2017, there is promotion column with integer type and value is 600
#
#
# Adding the Promotion column 
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, when

# Adding the Promotion column 
# Conditions = [(VechileAll["MOTIVE_POWER"] == 'ELECTRIC') & (VechileAll["FIRST_NZ_REGISTRATION_YEAR"] > 2016) ]
df_VehicleAll_New = df_VehicleAll.select(col("*"),when((col("MOTIVE_POWER") == "ELECTRIC") & (col("FIRST_NZ_REGISTRATION_YEAR") > 2016) \
                                               ,1).otherwise(0).alias("Promotion")   )
#
# Adding the EV_Flag column 
#Conditions = [(VechileAll["MOTIVE_POWER"] == 'ELECTRIC')]
df_VehicleAll_New = df_VehicleAll_New.select(col("*"),when((col("MOTIVE_POWER") == "ELECTRIC") \
                                               ,1).otherwise(0).alias("EV_Flag")   )

#This is reference in the reference section 9
#The running cost, the source data is from the Gen Less government website, Figure NZ website, 
#to get the EV vehicles equvilalent price of petrol and the average price of petrol from 2015-2019, 
#to calculate the % of EV running cost against tranditional vehicles. 
df_VehicleAll_New = df_VehicleAll_New.select(col("*") \
                                    ,when((col("EV_Flag") == 1)&(col("FIRST_NZ_REGISTRATION_YEAR") == 2015),0.9/1.83) \
                                    .when((col("EV_Flag") == 1)&(col("FIRST_NZ_REGISTRATION_YEAR") == 2016),0.9/1.73) \
                                    .when((col("EV_Flag") == 1)&(col("FIRST_NZ_REGISTRATION_YEAR") == 2017),0.9/1.88) \
                                    .when((col("EV_Flag") == 1)&(col("FIRST_NZ_REGISTRATION_YEAR") == 2018),0.9/2.01) \
                                    .when((col("EV_Flag") == 1)&(col("FIRST_NZ_REGISTRATION_YEAR") == 2019),0.9/1.93) \
                                    .otherwise(1).alias("RunningCost"))

# Testing the selection logic
print (df_VehicleAll_New.filter("EV_Flag = 1").head())

df_VehicleAll_EV = df_VehicleAll_New.filter("EV_Flag = 1")

#print ('Count of rows: {0}'.format(df_VehicleAll_New.count()))
#print ('Count of distinct rows: {0}'.format(df_VehicleAll_New.distinct().count()))

Row(ALTERNATIVE_MOTIVE_POWER=None, BASIC_COLOUR='RED', BODY_TYPE='HATCHBACK', CC_RATING=1, CHASSIS7=None, CLASS='MA', ENGINE_NUMBER='EM0003-7809930001', FIRST_NZ_REGISTRATION_YEAR=2018, FIRST_NZ_REGISTRATION_MONTH=3, GROSS_VEHICLE_MASS=Decimal('1060'), HEIGHT=0, IMPORT_STATUS='USED', INDUSTRY_CLASS='PRIVATE', INDUSTRY_MODEL_CODE='ZAA-451390', MAKE='SMART', MODEL='FORTWO', MOTIVE_POWER='ELECTRIC', MVMA_MODEL_CODE=None, NUMBER_OF_AXLES=2, NUMBER_OF_SEATS=2, NZ_ASSEMBLED='IMPORTED BUILT-UP', ORIGINAL_COUNTRY='GERMANY', POWER_RATING=0, PREVIOUS_COUNTRY='JAPAN', ROAD_TRANSPORT_CODE=None, SUBMODEL='ELECTRIC DRIVE', TLA=None, TRANSMISSION_TYPE=None, VDAM_WEIGHT=0, VEHICLE_TYPE='PASSENGER CAR/VAN', VEHICLE_USAGE='PRIVATE PASSENGER', VEHICLE_YEAR=2015, VIN11='WME4513902K', WIDTH=0, Promotion=1, EV_Flag=1, RunningCost=0.4477611940298508)


In [12]:
#2.3 Data exploration 
# Reverify the transformation

df_VehicleAll_EV.head(5)

print ('Count of rows: {0}'.format(df_VehicleAll_New.count()))
print ('Count of distinct rows: {0}'.format(df_VehicleAll_New.distinct().count()))


df_VehicleAll_New = df_VehicleAll_New.dropDuplicates()

df_VehicleAll_Sub = df_VehicleAll_New.select ("BASIC_COLOUR", "BODY_TYPE", "CC_RATING",
       "ENGINE_NUMBER", "FIRST_NZ_REGISTRATION_YEAR",
       "FIRST_NZ_REGISTRATION_MONTH",
       "IMPORT_STATUS", "MAKE",
       "MODEL", "MOTIVE_POWER", "NUMBER_OF_AXLES",
       "NUMBER_OF_SEATS", "NZ_ASSEMBLED", "ORIGINAL_COUNTRY", "POWER_RATING",
       "VEHICLE_TYPE",
       "VEHICLE_YEAR", "VIN11","EV_flag","Promotion","RunningCost").where("FIRST_NZ_REGISTRATION_MONTH < 2020")

df_VehicleAll_Sub.filter("EV_flag = 1").groupBy('EV_flag').mean().show()

print(df_VehicleAll_Sub.count())

df_VehicleAll_Sub.describe().show()
# Data Visualzation

df_VehicleAll_Sub.filter("EV_flag = 1").select("FIRST_NZ_REGISTRATION_YEAR","EV_flag").groupBy('FIRST_NZ_REGISTRATION_YEAR').sum('EV_flag').orderBy('FIRST_NZ_REGISTRATION_YEAR').show()

print(df_VehicleAll_Sub.count())

df_VehicleAll_Sub.printSchema()

# Get the missing data validation for each column, and all the columns missing value check pass over 90% 
df_VehicleAll_Sub.agg(*[(1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing') for c in df_VehicleAll_Sub.columns]).show()
#

Count of rows: 841295
Count of distinct rows: 839381
+-------+------------------+-------------------------------+--------------------------------+--------------------+--------------------+-----------------+-----------------+------------+-----------------+------------------+
|EV_flag|    avg(CC_RATING)|avg(FIRST_NZ_REGISTRATION_YEAR)|avg(FIRST_NZ_REGISTRATION_MONTH)|avg(NUMBER_OF_AXLES)|avg(NUMBER_OF_SEATS)|avg(POWER_RATING)|avg(VEHICLE_YEAR)|avg(EV_flag)|   avg(Promotion)|  avg(RunningCost)|
+-------+------------------+-------------------------------+--------------------------------+--------------------+--------------------+-----------------+-----------------+------------+-----------------+------------------+
|      1|43.229678123295145|             2018.2575013638843|                7.02924168030551|    1.69012547735952|   4.589198036006547|55.65390070921986|2016.998363338789|         1.0|0.951227495908347|0.5139562154969843|
+-------+------------------+-------------------------------

In [13]:
# Collective to visualization the sub dataset with columns
df_VehicleAll_Sub.head()
df_VehicleAll_Sub_EV = df_VehicleAll_Sub.filter(df_VehicleAll_Sub['EV_flag'] == 1).collect()
# df_VehicleAll_Sub_EV

Row(BASIC_COLOUR='BLACK', BODY_TYPE='STATION WAGON', CC_RATING=2199, ENGINE_NUMBER='D4HBEU142160', FIRST_NZ_REGISTRATION_YEAR=2015, FIRST_NZ_REGISTRATION_MONTH=4, IMPORT_STATUS='NEW', MAKE='HYUNDAI', MODEL='SANTA FE', MOTIVE_POWER='DIESEL', NUMBER_OF_AXLES=2, NUMBER_OF_SEATS=7, NZ_ASSEMBLED='IMPORTED BUILT-UP', ORIGINAL_COUNTRY='SOUTH KOREA', POWER_RATING=145, VEHICLE_TYPE='PASSENGER CAR/VAN', VEHICLE_YEAR=2015, VIN11='KMHSU81XSFU', EV_flag=0, Promotion=0, RunningCost=1.0)

#### Check the dataset inside relationship 
To identify which columns would be correlation with the Electric vehicles

In [14]:
# Check if the body corlour 
df_VehicleAll_Sub.filter("EV_flag = 1").select("BASIC_COLOUR","EV_flag").groupBy('BASIC_COLOUR').sum('EV_flag').orderBy('BASIC_COLOUR').show()

# Check if the YEAR 
df_VehicleAll_Sub.filter("EV_flag = 1").select("FIRST_NZ_REGISTRATION_YEAR","EV_flag").groupBy('FIRST_NZ_REGISTRATION_YEAR').sum('EV_flag').orderBy('FIRST_NZ_REGISTRATION_YEAR').show()



+------------+------------+
|BASIC_COLOUR|sum(EV_flag)|
+------------+------------+
|       BLACK|        1325|
|        BLUE|         899|
|       BROWN|          58|
|       CREAM|          89|
|        GOLD|         129|
|       GREEN|         102|
|        GREY|         867|
|      ORANGE|         226|
|      PURPLE|          11|
|         RED|         991|
|      SILVER|        1178|
|       WHITE|        3178|
|      YELLOW|         112|
+------------+------------+

+--------------------------+------------+
|FIRST_NZ_REGISTRATION_YEAR|sum(EV_flag)|
+--------------------------+------------+
|                      2015|         110|
|                      2016|         337|
|                      2017|        1859|
|                      2018|        2457|
|                      2019|        3581|
|                      2020|         821|
+--------------------------+------------+



### 4.2 Data Projected

In [15]:
df_VehicleAll_Sub.select("MOTIVE_POWER","VIN11").groupBy('MOTIVE_POWER').count().orderBy('MOTIVE_POWER').show()
#

df_VehicleAll_Sub = df_VehicleAll_Sub.select(col("*") \
                                    ,when((col("MOTIVE_POWER") == "ELECTRIC"),1) \
                                    .when((col("MOTIVE_POWER").like("%HYBRID%")),1) \
                                    .otherwise(2).alias("MOTIVE_POWER_GRP"))

df_VehicleAll_Sub.select("MOTIVE_POWER_GRP","VIN11").groupBy('MOTIVE_POWER_GRP').count().orderBy('MOTIVE_POWER_GRP').show()

+--------------------+------+
|        MOTIVE_POWER| count|
+--------------------+------+
|                 CNG|    22|
|              DIESEL|332152|
|DIESEL ELECTRIC H...|    20|
|       DIESEL HYBRID|    27|
|            ELECTRIC|  9165|
|ELECTRIC FUEL CEL...|     2|
|ELECTRIC [DIESEL ...|     2|
|ELECTRIC [PETROL ...|   338|
|                 LPG|   845|
|               OTHER|    38|
|              PETROL|478853|
|PETROL ELECTRIC H...|   206|
|       PETROL HYBRID| 14972|
|PLUGIN DIESEL HYBRID|    17|
|PLUGIN PETROL HYBRID|  2707|
+--------------------+------+

+----------------+------+
|MOTIVE_POWER_GRP| count|
+----------------+------+
|               1| 27114|
|               2|812252|
+----------------+------+



### 5. Data minging Method Selection
#Verify the final dataset 
Print the pari plot figures to help to identify which type of the function group
ML method will be applyed.
Based on the 
VechileAll_Sub = VechileAll[['CC_RATING','NUMBER_OF_SEATS','POWER_RATING','VEHICLE_YEAR','RunningCost','Promotion','EV_Flag']]



In [20]:
# Check the coorelation in the pyspark dataframe
print(df_VehicleAll_Sub.corr('EV_flag','Promotion'))

print(df_VehicleAll_Sub.corr('EV_flag','RunningCost'))

print(df_VehicleAll_Sub.corr('EV_flag','CC_RATING'))

0.9750464645194367
-0.9532461411091439
-0.14631986644837702


In [28]:
# Import the linear regression lib to process the dataset
import pyspark.mllib.stat as st
import numpy as np

from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
spark = SparkSession.builder.appName('logistic_regression_docs').getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.regression import LinearRegression

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

df_VehicleAll_Sub_DM01 = df_VehicleAll_Sub.select('CC_RATING',
       'NUMBER_OF_SEATS', 'POWER_RATING',
       'VEHICLE_YEAR','RunningCost','Promotion','EV_flag')


assembler = VectorAssembler(
    inputCols=['CC_RATING','NUMBER_OF_SEATS','POWER_RATING','VEHICLE_YEAR','RunningCost','Promotion','EV_flag'],
    outputCol='features')

Lr_assembler_output = assembler.transform(df_VehicleAll_Sub_DM01)
Lr_assembler_output.printSchema()

final_df_VehicleAll1 = Lr_assembler_output.select("features",'RunningCost')
final_df_VehicleAll2 = Lr_assembler_output.select("features",'Promotion')
final_df_VehicleAll3 = Lr_assembler_output.select("features",'EV_flag')
final_df_VehicleAll1.show()
final_df_VehicleAll2.show()
final_df_VehicleAll3.show()

root
 |-- CC_RATING: integer (nullable = true)
 |-- NUMBER_OF_SEATS: integer (nullable = true)
 |-- POWER_RATING: integer (nullable = true)
 |-- VEHICLE_YEAR: integer (nullable = true)
 |-- RunningCost: double (nullable = false)
 |-- Promotion: integer (nullable = false)
 |-- EV_flag: integer (nullable = false)
 |-- features: vector (nullable = true)

+--------------------+-----------+
|            features|RunningCost|
+--------------------+-----------+
|[2199.0,7.0,145.0...|        1.0|
|[2378.0,5.0,100.0...|        1.0|
|[1190.0,5.0,0.0,2...|        1.0|
|[3564.0,5.0,210.0...|        1.0|
|[1496.0,5.0,97.0,...|        1.0|
|[1986.0,5.0,107.0...|        1.0|
|[2776.0,7.0,147.0...|        1.0|
|[2354.0,5.0,148.0...|        1.0|
|[1997.0,5.0,135.0...|        1.0|
|[2982.0,5.0,126.0...|        1.0|
|[1598.0,5.0,100.0...|        1.0|
|[2982.0,5.0,126.0...|        1.0|
|[1999.0,5.0,129.0...|        1.0|
|[2442.0,5.0,135.0...|        1.0|
|[2696.0,7.0,121.0...|        1.0|
|[1299.0,5.0,63.

In [31]:
# Split the dataset into training and test sets, this is testing via the running cost 
from pyspark.ml.regression import LinearRegression
train_data,test_data = final_df_VehicleAll1.randomSplit([0.7,0.3])

#
lr_vehicles_all1 = LinearRegression(labelCol='RunningCost')
lrModel_vehicles1 = lr_vehicles_all1.fit(train_data)

print("Coefficients: {} Intercept: {}".format(lrModel_vehicles1.coefficients,lrModel_vehicles1.intercept))

Coefficients: [-2.0269068690384315e-20,-1.3032967738708907e-17,1.7580139850665714e-18,5.387723885921349e-17,1.0000000000003049,-6.30497048871144e-16,1.4856647414433639e-13] Intercept: -4.136515715834219e-13


In [32]:
# Checking the result of the test dataset
test_results = lrModel_vehicles1.evaluate(test_data)

# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()

# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))

print("R2: {}".format(test_results.r2))

+--------------------+
|           residuals|
+--------------------+
|2.220446049250313...|
|2.220446049250313...|
|                 0.0|
|2.220446049250313...|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|                 0.0|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
|                 0.0|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
|2.220446049250313...|
+--------------------+
only showing top 20 rows

RSME: 4.780251196353075e-15
R2: 1.0
